In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
#MorphoMolNotebooks.get_low_energy_and_theta_states("../../Data/hpc_out/op/3_6r7m/1/", "6r7m")
#@load "../../Data/hpc_out/rwm_ma_3_6r7m/7_rwm_ma_3_6r7m.jld2" input output
#minimum(get_theta(input, output))

In [5]:
mol_type = "6diy"
algo = "hmc"
energy = "ma"
n_mol = 12

#folder = "../../Data/hpc_out/$energy/3_6r7m/3/"
#folder = "../../Data/hpc_out/$energy/3_6r7m/refine_1_71/2/"
folder = "../../Data/hpc_out/$(algo)_$(energy)_$(n_mol)_$(mol_type)/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
thetas = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            # if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
            #     println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
            #     continue
            # end

            id = parse(Int, split(file, separator)[id_index])
            theta = MorphoMolNotebooks.get_theta(input, output)
            string = "$(id): $(input["T"]) |$(output["αs"][end]) | $(minimum(output["Es"])) | $(theta)"

            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
            thetas[id] = theta
        catch e
            println(e)
            println("Error in $file")
        end
    end
end
println("Minimal energy configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")
println("Minimal theta configuration id: $(argmin(thetas))")
for elem in evaluation_strings
    println(elem)
end

EOFError()
Error in 85_6diy_protor_oj_0_0_os_1_1.jld2
Minimal energy configuration id: 67 | Average acceptance rate: 0.6311808204602288
Minimal theta configuration id: 1
1: 1.4 |0.5923004 | 2029.281564006774 | Inf
2: 1.4 |0.6305165 | 2037.9517936282114 | Inf
3: 1.4 |0.65377295 | 2041.3226765850984 | Inf
4: 1.4 |0.6239866 | 2040.586763508815 | Inf
5: 1.4 |0.6578203 | 2047.2310419632609 | Inf
6: 1.4 |0.6654043 | 2024.46019168087 | Inf
7: 1.4 |0.65832686 | 2046.4143570056847 | Inf
8: 1.4 |0.649994 | 2022.3663290782406 | Inf
9: 1.4 |0.6033318 | 2048.697903775345 | Inf
10: 1.4 |0.67388546 | 2049.2473545371486 | Inf
11: 1.4 |0.62569565 | 2034.1905750744684 | Inf
12: 1.4 |0.65383726 | 2042.0433819714463 | Inf
13: 1.4 |0.68564844 | 2026.4594352254778 | Inf
14: 1.4 |0.6250473 | 2035.49396721924 | Inf
15: 1.4 |0.69556653 | 2044.923028091938 | Inf
16: 1.4 |0.5596574 | 2020.0195342400125 | Inf
17: 1.4 |0.6524474 | 2029.277773567472 | Inf
18: 1.4 |0.6078984 | 2031.7908604389788 | Inf
19: 1.4 |0.605

In [20]:
[i for i in 1:length(min_Es) if min_Es[i] < minimum(min_Es) * 1.0025 && thetas[i] > 5.0]

41-element Vector{Int64}:
  1
  2
  4
  6
  9
 12
 14
 17
 21
 25
 26
 27
 31
  ⋮
 65
 71
 76
 82
 83
 86
 91
 92
 93
 96
 98
 99

In [21]:
[i for i in 1:length(thetas) if thetas[i] < 4.0]

Int64[]

In [22]:
for file in readdir(folder)
    try
        if split(file, ".")[end] == "jld2"
            @load "$folder$file" input output
            mindex = argmin(output["Es"])
            state = output["states"][mindex]
            id = parse(Int, split(file, "_")[id_index])
            radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
            colors = [MorphoMolNotebooks.STANDARD_COLORS[i] for i in 1:input["n_mol"] for _ in 1:length(input["template_radii"])]
            MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_point_vector_realization(state, input["template_centers"]), radii, colors, "$(folder)$(id)")
        end
    catch e
        println(e)
        continue
    end
end

In [14]:
@load "$(folder)/31_$(algo)_$(energy)_$(n_mol)_$(mol_type).jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [9]:
@load "../../Data/hpc_out/$energy/3_6r7m/1/71_$(algo)_$(energy)_$(n_mol)_$(mol_type).jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [10]:
#min_dex = argmin(output["Es"])
min_dex = 9800
state_string = "["
for e in output["states"][min_dex]
    state_string *= "$(e),"
end
state_string *= "]"
state_string

"[1.509228153653038,2.075751345239662,-5.318232594141871,56.23954638527144,101.57490490478578,127.98448407280469,-24.01180611567413,43.65738040984568,-7.369426210947061,56.45760461480674,80.18572074318053,140.3786636714549,-12.855383069920308,8.616148889482243,10.715234961128793,48.026972725414986,119.90393247014737,132.87922126233056,]"

# Print Single Assembly Path

In [4]:
@load "$(folder)/11_$(algo_e)_2_$(mol_type).jld2" input output

LoadError: UndefVarError: `algo_e` not defined

In [ ]:
"$(folder)/11_$(algo_e)_2_$(mol_type).jld2"

"../../Data/hpc_out/2_6r7m/rwm_ma_2_6r7m//11_rwm_ma_2_6r7m.jld2"

In [ ]:
using GeometryBasics

In [ ]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [ ]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
